In [ ]:
from pathlib import Path
import rioxarray as riox
import xarray as xr
import os


def open_raster(
    filename: str | os.PathLike, mask_and_scale: bool = True, **kwargs
) -> xr.DataArray | xr.Dataset:
    """Open a raster dataset using rioxarray."""
    ds = riox.open_rasterio(filename, mask_and_scale=mask_and_scale, **kwargs)

    if isinstance(ds, list):
        raise ValueError("Multiple files found.")

    return ds

def read_product_trait_map(
    trait_id: str, path: Path, band: int | None = None
) -> xr.DataArray | xr.Dataset:
    """Get the path to a specific trait map."""
    
    path = Path(path)
    
    # If path is a .tif file, use it directly
    if path.suffix == ".tif":
        fn = path
    else:
        # Otherwise, assume it's a directory and build the filename
        fn = path / f"{trait_id}.tif"

    if band is not None:
        return open_raster(fn).sel(band=band)
    return open_raster(fn)

def read_product_tiff_to_df(trait, path, band=1):
    return read_product_trait_map(trait_id=trait, path=path, band=band).to_dataframe(name=trait).drop(columns=["band", "spatial_ref"]).dropna()
        



## Some of the product traits maps are in half-degree reoslution, we use the below script to project them to 1 degree

In [ ]:
height = 'Plant.Height_05deg_expln.tif'

In [5]:
import rioxarray
import xarray as xr
import os
import rioxarray as riox
from rasterio.enums import Resampling
def interpolate_halfdeg_to_1deg(raster_path: str | os.PathLike) -> xr.DataArray:
    """Interpolate 0.5-degree resolution data to 1-degree resolution."""
    ds = riox.open_rasterio(raster_path, mask_and_scale=True)
    print(ds.rio.crs)  # Check the CRS of the dataset
    # Reproject to same CRS but with coarser resolution (1°)
    ds_1deg = ds.rio.reproject(ds.rio.crs, resolution=(1.0, 1.0), resampling=Resampling.average)
    return ds_1deg


In [ ]:
# Step 3: Save to new GeoTIFF
height_1_deg = interpolate_halfdeg_to_1deg(height)
